In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import json

def process_titles_descriptions_file(file_path, processed_data):
    #Lê um arquivo JSON do nosso dataset, processa cada notícia para formatar conforme o solicitado e adiciona à lista processed_data
    with open(file_path, 'r', encoding='utf-8') as file:
        json_data = [json.loads(line) for line in open(file_path, 'r')]

        count = 0
        limit = 100000 #pegando as primeiras 10 mil linhas primeiro para testar

        for item in json_data:

            if count >= limit:
                break

            title = item["title"]
            description = item["content"]
            formatted_text = f"CREATE A DESCRIPTION FOR THIS TITLE.\n[|Title|] {title}[|eTitle|]\n\n[|description|]{description}[|eDescription|]"

            if description and description.strip():
              processed_data.append({"input": formatted_text})
              count += 1

              if count >= limit:
                break

In [3]:
# Lista para armazenar todos os dados processados
processed_data = []

# Adicionar dados processados do arquivo JSON à lista
process_titles_descriptions_file(r'drive/MyDrive/COLAB/trn.json', processed_data)

# Salvar todos os dados processados em um arquivo JSON
output_filename = r'drive/MyDrive/COLAB/titles_dataset_chat_data.json'
with open(output_filename, 'w', encoding='utf-8') as file:
    json.dump(processed_data, file, ensure_ascii=False, indent=4)

print(f"Todos os dados reformatados foram salvos em '{output_filename}'.")
print(f"Exemplo de processed_data: {processed_data[:5]}")  # Mostra os primeiros 5 exemplos para checar

Todos os dados reformatados foram salvos em 'drive/MyDrive/COLAB/titles_dataset_chat_data.json'.
Exemplo de processed_data: [{'input': 'CREATE A DESCRIPTION FOR THIS TITLE.\n[|Title|] Girls Ballet Tutu Neon Pink[|eTitle|]\n\n[|description|]High quality 3 layer ballet tutu. 12 inches in length[|eDescription|]'}, {'input': "CREATE A DESCRIPTION FOR THIS TITLE.\n[|Title|] Mog's Kittens[|eTitle|]\n\n[|description|]Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.[|eDescription|]"}, {'input': 'CREATE A DESCRIPTION FOR THIS TITLE.\n[|Title|] Girls Ballet Tutu Neon Blue[|eTitle|]\n\n[|description|]Dance tutu for girls ages 2-8 years. P

In [4]:
!python -m pip install --upgrade pip
!pip install triton
!pip install xformers
!pip install transformers
!pip install torch

In [5]:
!conda create --name unsloth_env \
    python=3.11 \
    pytorch-cuda=12.1 \
    pytorch cudatoolkit xformers -c pytorch -c nvidia -c xformers \
    -y
!conda activate unsloth_env

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes

/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-c5hjmdmr/unsloth_f81bcb87c9e945e0b6b5b8aebad4e07f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-c5hjmdmr/unsloth_f81bcb87c9e945e0b6b5b8aebad4e07f
  Resolved https://github.com/unslothai/unsloth.git to commit c3f4e9a87d964ecee1efd9963f497119edbefaab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "drive/MyDrive/COLAB/titles_dataset_chat_data.json"
OUTPUT_PATH_DATASET = "drive/MyDrive/COLAB/formatted_titles_dataset_chat_data.json"

# Reduzir o tamanho máximo de sequência
max_seq_length = 1024  # Tamanho reduzido para economizar memória
dtype = torch.float16  # Usar float16 para otimizar o uso de memória
load_in_4bit = True

# Tentar um modelo menor primeiro para evitar problemas de memória
model_name = "unsloth/mistral-7b-bnb-4bit"  # Modelo menor com quantização 4-bit

# Carregar o modelo com offloading automático entre CPU e GPU
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,  # Modelo escolhido
    max_seq_length=max_seq_length,  # Sequência ajustada
    dtype=dtype,  # Usando float16
    load_in_4bit=load_in_4bit,  # Quantização 4-bit
    device_map="auto"  # Mapeamento automático entre CPU e GPU
)

print("Modelo carregado com sucesso!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post3: Fast Mistral patching. Transformers = 4.45.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

Modelo carregado com sucesso!


In [7]:
def format_dataset_into_model_input(data):
    def separate_text(full_text):
        title_start = full_text.find("[|Title|]") + len("[|Title|]")
        title_end = full_text.find("[|eTitle|]")
        description_start = full_text.find("[|description|]") + len("[|description|]")
        description_end = full_text.find("[|eDescription|]")

        instruction = full_text.split('\n')[0]
        input_text = full_text[title_start:title_end].strip()
        response = full_text[description_start:description_end].strip()

        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset
    for prompt in data['train']['input']:
        instruction, input_text, response = separate_text(prompt)
        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(response)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

In [8]:
data_from_json = load_dataset("json", data_files="drive/MyDrive/COLAB/titles_dataset_chat_data.json")
format_dataset_into_model_input(data_from_json)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset salvo em drive/MyDrive/COLAB/formatted_titles_dataset_chat_data.json


In [9]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post3: Fast Mistral patching. Transformers = 4.45.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9.post3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [11]:
from datasets import load_dataset

OUTPUT_PATH_DATASET = "drive/MyDrive/COLAB/formatted_titles_dataset_chat_data.json"

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
print(dataset[:5])

{'instruction': ['CREATE A DESCRIPTION FOR THIS TITLE.', 'CREATE A DESCRIPTION FOR THIS TITLE.', 'CREATE A DESCRIPTION FOR THIS TITLE.', 'CREATE A DESCRIPTION FOR THIS TITLE.', 'CREATE A DESCRIPTION FOR THIS TITLE.'], 'input': ['Girls Ballet Tutu Neon Pink', "Mog's Kittens", 'Girls Ballet Tutu Neon Blue', 'The Prophet', 'Rightly Dividing the Word'], 'output': ['High quality 3 layer ballet tutu. 12 inches in length', 'Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.', 'Dance tutu for girls ages 2-8 years. Perfect for dance practice, recitals and performances, costumes or just for fun!', 'In a distant, timeless place, a mysteriou

In [13]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [14]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 20,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 20
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.344000
2,2.339300
3,2.204500
4,2.048600
5,2.017000
6,1.710900
7,1.876300
8,1.366100
9,1.542200
10,1.698700


In [16]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "CREATE A DESCRIPTION FOR THIS TITLE",
        "The Lord Of The Rings.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nCREATE A DESCRIPTION FOR THIS TITLE\n\n### Input:\nThe Lord Of The Rings.\n\n### Response:\nThe Lord of the Rings is a trilogy of high fantasy novels written by English author J. R. R. Tolkien. The trilogy, which is set in the fictional world of Middle-earth, follows the story of the hobbit Frodo Baggins and the w']

In [17]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "CREATE A DESCRIPTION FOR THIS TITLE",
        "The Little Prince", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nCREATE A DESCRIPTION FOR THIS TITLE\n\n### Input:\nThe Little Prince\n\n### Response:\n"The Little Prince is the most translated book in the world, and the most widely read and reread book in the French language. It is a classic of children\'s literature, but it is much more than that. It is a classic of world literature, a book that speaks to the child in all of']

In [18]:
model.save_pretrained("drive/MyDrive/COLAB/fine-tuning-fiap/lora_model") # Local saving
tokenizer.save_pretrained("drive/MyDrive/COLAB/fine-tuning-fiap/lora_model")

('drive/MyDrive/COLAB/fine-tuning-fiap/lora_model/tokenizer_config.json',
 'drive/MyDrive/COLAB/fine-tuning-fiap/lora_model/special_tokens_map.json',
 'drive/MyDrive/COLAB/fine-tuning-fiap/lora_model/tokenizer.model',
 'drive/MyDrive/COLAB/fine-tuning-fiap/lora_model/added_tokens.json',
 'drive/MyDrive/COLAB/fine-tuning-fiap/lora_model/tokenizer.json')

In [19]:
import json
titles_test = []

json_data = [json.loads(line) for line in open('drive/MyDrive/COLAB/tst.json', 'r')]
titles_test = [item['title'] for item in json_data]

In [20]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "CREATE A DESCRIPTION FOR THIS TITLE",
        titles_test[15], # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nCREATE A DESCRIPTION FOR THIS TITLE\n\n### Input:\nNoddy Gets Into Trouble\n\n### Response:\nNoddy is a happy little boy who lives in Toyland. He has a dog called Bumpy and a cat called Big Ears. He also has a car called Golly. Noddy is always getting into trouble.</s>']

In [21]:
import torch
print(torch.cuda.is_available())  # Deve retornar True se a GPU estiver ativa

True


In [22]:
!nvidia-smi

Sun Sep 29 01:16:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0              30W /  70W |   6263MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [23]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "drive/MyDrive/COLAB/fine-tuning-fiap/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.9.post3: Fast Mistral patching. Transformers = 4.45.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will load drive/MyDrive/COLAB/fine-tuning-fiap/lora_model as a legacy tokenizer.


In [24]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
[
alpaca_prompt.format(
        "CREATE A DESCRIPTION FOR THIS TITLE",
        titles_test[1],
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<s>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
CREATE A DESCRIPTION FOR THIS TITLE

### Input:
Why Don't They Just Quit? DVD Roundtable Discussion: What Families and Friends need to Know About Addiction and Recovery

### Response:
"This is a great resource for families and friends of addicts. It is a must-have for anyone who wants to understand addiction and recovery."--Michael J. O'Donnell, PhD, LCSW, author of The Addiction Recovery Workbook</s>
